In [1]:
rm(list=ls())
pacman::p_load(data.table, MASS, ggplot2, stringr, matrixStats, doParallel, abind, ncdf4)

parallel::detectCores()
## Resizing notebook plot space
options(repr.plot.width=16, repr.plot.height=9)

[1] 48

In [2]:
## A function to just open the ncdf file and it's corresponding metadata

ncdf_open_extract_dims <- function(file) {
    
    le_file <- list()
    
    ## Open the file
    le_file[[1]] <- nc_open(paste0(file))
    
    
    ## Get all dimnames of the xarray
    dim_names <- names(le_file[[1]]$dim)

    ## Get the corresponding dimvalues
    dim_vals <- lapply(dim_names, function(x) {
        return(le_file[[1]]$dim[[paste0(x)]]$vals)
    })

    names(dim_vals) <- dim_names
    
    le_file[[2]] <- dim_vals
    
    return(le_file)
    
}

In [ ]:
file <- paste0("/ihme/forecasting/data/fbd_scenarios_data/forecast/death/20170322_gk_scenario/cvd_ihd.nc")
example_nc <- ncdf_open_extract_dims(file)
example_nc[[1]]

In [ ]:
## Now, here's the important part: subsetting and getting just the data we need, and also to make sure that, 
## if a subset occurs, then we don't exceed the dimensions in the count option

lengths(example_nc[[2]])
example_nc[[2]]["location_id"]

In [ ]:
## So, the first datapoint is scenario == -1. Let's extract that
system.time(edu_pes <- ncvar_get(gpr_ncopen, "__xarray_dataarray_variable__", start=c(1,1,1,1,1,1), 
                                 count = c(2,2,2,2,2,2)))
str(edu_pes)

## Melt to a data table

In [ ]:
## Create a grid of the coordinates 

# Name of all dimensions
dims <- names(gpr_ncopen$dim)

# List of dim values for each of the above names
dim_list <- lapply(dims, function(x) {
    gpr_ncopen$dim[[paste0(x)]]$vals
})

names(dim_list) <- dims

In [ ]:
## the dimnames list has to be of the EXACT dimensions as that of the dimension of the array!
dimnames_smaller <- lapply(dim_names, function(x) {dim_vals[[paste0(x)]][1:2]} )
dimnames_smaller

In [ ]:
attributes(edu_pes)$dimnames <- dimnames_smaller